In [ ]:
!pip install tensorflow=='2.6.0'

In [2]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import six
import time

import imageio
import glob
import scipy
import numpy as np
#import cv2
import csv
import scipy.misc
import pandas as pd
import lxml.etree as ET


import pathlib
import hashlib

from PIL import Image
from six import BytesIO
from IPython.display import display
%matplotlib inline

In [3]:
os.chdir('/content/drive/MyDrive/wobot_final/')

In [4]:
if 'models' in pathlib.Path.cwd().parts:
  while 'models' in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [31]:
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [32]:
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [8]:
labelmap_path = '/content/drive/MyDrive/wobot_final/models/research/object_detection/data/tf_label_map.pbtxt'

In [9]:
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)

In [ ]:

tf.keras.backend.clear_session()
model = tf.saved_model.load('/content/drive/MyDrive/wobot_final/models/research/object_detection/inference_graph/saved_model')

In [33]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  if image.getdata().mode == "RGBA":
    image = image.convert('RGB')
  np_array = np.array(image.getdata())
  reshaped = np_array.reshape((im_height, im_width, 3))
  return reshaped.astype(np.uint8)

In [34]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [14]:
os.chdir('/content/drive/MyDrive/wobot_final/New Folder/')

In [27]:
def csv_to_xml(image_file):
  df1 = pd.DataFrame()
  annot_file = image_file[53:-3]+'xml' 
  try:
   open(f'annotations/{annot_file}', 'x')
  except FileExistsError:
    pass
  df1 = pd.read_csv(f'csvS/{image_file[53:-3]}csv')
  data = ET.Element('annotations')
  item1 = ET.SubElement(data, 'folder')
  item2 = ET.SubElement(data, 'filename')
  item1.text = 'images'
  item2.text = f'{image_file[53:]}'
  for i in df1.itertuples():
    item = ET.SubElement(data, 'object')
    class_name = ET.SubElement(item, 'name')
    bounding_box = ET.SubElement(item, 'bndbox')
    x_min = ET.SubElement(bounding_box, 'xmin')
    y_min = ET.SubElement(bounding_box, 'ymin')
    x_max = ET.SubElement(bounding_box, 'xmax')
    y_max = ET.SubElement(bounding_box, 'ymax')
    class_name.text = i[6]
    x_min.text = str(i[2])
    y_min.text = str(i[3])
    x_max.text = str(i[4])
    y_max.text = str(i[5])



  mydata = ET.tostring(data, pretty_print = True, xml_declaration = True, encoding = 'utf-8')
  x = open(f'annotations/{annot_file}', 'wb')
  x.write(mydata)
  

In [35]:
color = {'Red':(255,0,0), 'Blue':(0,0,255), 'Green':(0,255,0),'Yellow':(255,255,0), 'White':(255,255,255), 'Orange':(255,102,0)}

In [36]:
from sklearn.cluster import KMeans
from scipy.spatial import distance

In [48]:
def color_detection(image):
  image_array = np.asarray(image)
  
  means = {}
  #scale_percent = 2000
  #width = int((image.shape[1]) * (scale_percent/100))
  #height = int((image.shape[0])*(scale_percent/100))
  sr = cv2.dnn_superres.DnnSuperResImpl_create()
  path = "ESPCN_x3.pb"
  model_name = 'espcn'
  model_Scale = 3

  sr.readModel(path)
  sr.setModel(model_name, model_Scale)
  result_img = sr.upsample(image_array)
  #print(result_img)
  img1=result_img.reshape((result_img.shape[1]*result_img.shape[0],3)) #flatten images
  
  #dim = (width, height)
  #resized_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
  #resized_image.reshape((resized_image.shape[1]*resized_image.shape[0],3))
  kmeans = KMeans(n_clusters = 3)
  kmeans.fit(img1)
  centroids = kmeans.cluster_centers_
  centroid_n = centroids.mean(axis = 0)
  for i in color.keys():
    dist = distance.euclidean(centroid_n, color[i])
    means[i] = dist
  return color[min(means, key=means.get)]

In [ ]:
!pip install opencv-contrib-python

In [30]:
for image_path in glob.glob('/content/drive/MyDrive/wobot_final/New Folder/images/*.png'):
  output = []
  df = pd.DataFrame()
  image_np = load_image_into_numpy_array(image_path)
  output_dict = run_inference_for_single_image(model, image_np)
  dets = output_dict['detection_boxes']
  classes = output_dict['detection_classes']
  scores = output_dict['detection_scores']
  im_height = image_np.shape[0]
  im_width = image_np.shape[1]
  labels = ['Helmet', 'Head', 'Person']

  for i in range(len(dets[0])):
    if (scores[i] > 0.2) and (len(classes) != 0):
      y1, x1, y2, x2 = dets[i,:]
      left, right, top, bottom = x1 * im_width, x2 * im_width, y1 * im_height, y2 * im_height
      #img = Image.fromarray(image_np)
      #cropped_image = img.crop((int(left),int(top),int(right),int(bottom)))
      #bb_color = color_detection(cropped_image)
      cv2.rectangle(image_np, (int(left),int(top)), (int(right),int(bottom)), (36,255,12), 2)
      cv2.putText(image_np, str(labels[int(classes[i])-1])+":"+str(int(scores[i]*100)), (int(left), int(top-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
      output=[image_path[53:], int(left),int(top), int(right),int(bottom), str(labels[int(classes[i])-1]) ]
      #print(output)
      s = pd.Series(output, index=None)
      df = df.append(s, ignore_index= True)
  df.to_csv(f'csvS/{image_path[53:-3]}csv', index=None)
  cv2.imwrite(f'result/annot_{image_path[53:-3]}png', image_np)

In [ ]:
for image_file in glob.glob('/content/drive/MyDrive/wobot_final/New Folder/images/*.png'):
  csv_to_xml(image_file)

In [38]:
os.chdir('/content/drive/MyDrive/wobot_final/video/')

In [33]:
def video_inference1(video_file):
  cap = cv2.VideoCapture(video_file)
  out_path = os.path.join('/content/drive/MyDrive/wobot/video/', 'output1.webM')
  size = (1280,720)
  fps = 10
  fourcc = cv2.VideoWriter_fourcc('V','P','8','0')
  out = cv2.VideoWriter(out_path,fourcc, fps, size)
  labels = ['Helmet', 'Head', 'Person']

  while cap.isOpened():
    ret, image_np = cap.read()
    if not ret:
      break
    output_dict = run_inference_for_single_image(model, image_np)
    dets = output_dict['detection_boxes']
    classes = output_dict['detection_classes']
    scores = output_dict['detection_scores']
    im_height = image_np.shape[0]
    im_width = image_np.shape[1]


    for i in range(len(dets[0])):
      if (scores[i] > 0.2) and (len(classes) != 0):
        y1, x1, y2, x2 = dets[i,:]
        left, right, top, bottom = x1 * im_width, x2 * im_width, y1 * im_height, y2 * im_height
        #img = Image.fromarray(image_np)
        #cropped_image = img.crop((int(left),int(top),int(right),int(bottom)))
        #bb_color = color_detection(cropped_image)
        cv2.rectangle(image_np, (int(left),int(top)), (int(right),int(bottom)),(36,255,12), 2)
        cv2.putText(image_np, str(labels[int(classes[i])-1])+":"+str(int(scores[i]*100)), (int(left), int(top-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    im_vid = cv2.resize(image_np,(1280,720))
    out.write(im_vid)
  out.release()





In [34]:
video_inference1('input.mp4')

In [49]:
def video_inference(video_file):
  cap = cv2.VideoCapture(video_file)
  out_path = os.path.join('/content/drive/MyDrive/wobot_final/video/', 'output.webM')
  size = (1280,720)
  fps = 10
  fourcc = cv2.VideoWriter_fourcc('V','P','8','0')
  out = cv2.VideoWriter(out_path,fourcc, fps, size)
  labels = ['Helmet', 'Head', 'Person']

  while cap.isOpened():
    ret, image_np = cap.read()
    if not ret:
      break
    output_dict = run_inference_for_single_image(model, image_np)
    dets = output_dict['detection_boxes']
    classes = output_dict['detection_classes']
    scores = output_dict['detection_scores']
    im_height = image_np.shape[0]
    im_width = image_np.shape[1]


    for i in range(len(dets[0])):
      if (scores[i] > 0.2) and (len(classes) != 0):
        y1, x1, y2, x2 = dets[i,:]
        left, right, top, bottom = x1 * im_width, x2 * im_width, y1 * im_height, y2 * im_height
        img = Image.fromarray(image_np)
        cropped_image = img.crop((int(left),int(top),int(right),int(bottom)))
        bb_color = color_detection(cropped_image)
        cv2.rectangle(image_np, (int(left),int(top)), (int(right),int(bottom)), bb_color, 2)
        cv2.putText(image_np, str(labels[int(classes[i])-1])+":"+str(int(scores[i]*100)), (int(left), int(top-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    im_vid = cv2.resize(image_np,(1280,720))
    out.write(im_vid)
  out.release()





In [40]:
from scipy.spatial import distance
from PIL import Image, ImageDraw, ImageFont
from sklearn.cluster import KMeans
import time
import scipy.misc

In [47]:
import cv2

In [46]:
!pip install opencv-python

In [43]:
!pip install ffmpeg-python

In [ ]:
video_inference('input.mp4')

In [44]:
import ffmpeg

In [45]:
!ffmpeg -i  'output1.webM' -vf scale=1280:720 'output_1.mp4' 

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [52]:
!ffmpeg -i  'output.webM' -vf scale=1280:720 'output_2.mp4' 

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib